How to deal with multiple related constants?

```
STATUS_STARTED = 'started'
STATUS_PENDING = 'pending'
STATUS_OK = 'ok'
```

We want an immutable collection of related constant members, that have unique names, and an associated constant value.

__Alias__

Sometime we want multiple symbols to refer to the same 'thing'

```
POLY_4 = 4
RECTANGLE = 4
SQUARE = 4
RHOMBUS = 4
```

These are all unique symbols, but point to the same value.

### Using Enumerations

Introduced in Python 3.4, enumerations are available in the `enum` module and can be created by subclassing `Enum`.

In [16]:
import enum

class Color(enum.Enum):
    RED = 1
    GREEN = 2
    BLUE = 3

In [3]:
# The type of a member is the class
isinstance(Color.RED, Color)

True

In [8]:
# A member and its associated value are NOT equal
Color.RED == 1

False

In [18]:
# Members have attributes for their name and value
Color.RED.name, Color.RED.value

('RED', 1)

In [10]:
# Members are hashable
pixel_color = {
    Color.RED: 100,
    Color.GREEN: 25,
    Color.BLUE: 255
}

In [13]:
# Accessing member by value
print(Color(2))

Color['BLUE']

Color.GREEN


<Color.BLUE: 3>

In [19]:
# Enumerations are iterables and definition order is preserved
for color in Color:
    print(color)

Color.RED
Color.GREEN
Color.BLUE


__Constant Members and Constant Values__

Once an enumeration has been declared:
- you cannot add or remove members
- member values are immutable
- cannot be subclassed (unless it contains no members)

In [20]:
Color.__members__

mappingproxy({'RED': <Color.RED: 1>,
              'GREEN': <Color.GREEN: 2>,
              'BLUE': <Color.BLUE: 3>})

### Aliases

Enumerations are guaranteed to have only unique members, but we can stil define multiple members that hold the same value. Any members with duplicate values will point to the firstly defined member with that value, they are simply just aliases.

In [21]:
class Color(enum.Enum):
    red = 1
    crimson = 1
    carmine = 1
    blue = 2
    aquamarine = 2

In [22]:
# If we iterate Color we only have the 'base' members
list(Color)

[<Color.red: 1>, <Color.blue: 2>]

In [23]:
print(Color.crimson)

Color.crimson is Color.red

Color.red


True

In [24]:
print(Color(1))
Color['crimson']

Color.red


<Color.red: 1>

In [25]:
# We can see all members, even aliases, through the dunder members property
Color.__members__

mappingproxy({'red': <Color.red: 1>,
              'crimson': <Color.red: 1>,
              'carmine': <Color.red: 1>,
              'blue': <Color.blue: 2>,
              'aquamarine': <Color.blue: 2>})

In [26]:
# You can restrict enumerations from having aliases
@enum.unique
class Color(enum.Enum):
    red = 1
    crimson = 1
    carmine = 1
    blue = 2
    aquamarine = 2

ValueError: duplicate values found in <enum 'Color'>: crimson -> red, carmine -> red, aquamarine -> blue

We can use enumerations to map status messages (from different services for example) to our own status message.

In [33]:
class Status(enum.Enum):
    ready = 1
    
    running = 2
    busy = 2
    processing = 2
    
    ok = 3
    finished_no_error = 3
    success = 3
    
    errors = 4
    finished_with_error = 4

In [35]:
import json

payload = json.loads('''
{
    "status": "success"
}
''')

my_status = Status[payload['status']].name
print(my_status)

ok


### Customizing/Extending Enumerations

Since Enums are classes, we can define functions for that class. And because Enum members become instances of the class, we can have bound methods for that instance. 

We can also define dunder methods, such as `__str__, __eq__, __lt__` etc.

In [37]:
# By default, every member of an enum is truthy irrespective of value,
# but we can override that.
class State(enum.Enum):
    READY = 1
    BUSY = 0
    
    def __bool__(self):
        return bool(self.value)

if not State.BUSY:
    print('do work')

do work


In [41]:
# We cannot subclass enumerations that define members, but we can subclass
# enums that define methods for example

class OrderedEnum(enum.Enum):
    def __lt__(self, other):
        if isinstance(other, OrderedEnum):
            return self.value < other.value
        else:
            return NotImplemented
        
class Number(OrderedEnum):
    ONE = 1
    TWO = 2
    THREE = 3

In [42]:
Number.ONE < Number.TWO

True

In [46]:
class HTTPStatus(enum.Enum):
    OK = (200, 'Request Successful')
    NOT_FOUND = (404, 'Resource Not Found')
    
    def __new__(cls, value, phrase):
        member = object.__new__(cls)
        member._value_ = value
        member.phrase = phrase
        return member

    
HTTPStatus.OK.value, HTTPStatus.OK.name, HTTPStatus.OK.phrase

(200, 'OK', 'Request Successful')

In [47]:
HTTPStatus(200)

<HTTPStatus.OK: 200>

### Automatic Values

Introduced in Python 3.6, we can use `enum.auto()` to automatically generate member values. The default implementation uses sequential integer numbers.

In [48]:
import enum

In [49]:
class State(enum.Enum):
    WAITING = enum.auto()
    STARTED = enum.auto()
    FINISHED = enum.auto()

In [51]:
for member in State:
    print(member.name, member.value)

WAITING 1
STARTED 2
FINISHED 3


In [55]:
class State(enum.Enum):
    def _generate_next_value_(name, start, count, last_values):
        print(name, start, count, last_values)
        return name.title()
    
    a = enum.auto()
    b = enum.auto()
    c = enum.auto()

a 1 0 []
b 1 1 ['A']
c 1 2 ['A', 'B']


In [57]:
for mem in State:
    print(mem.name, mem.value)

a A
b B
c C


In [59]:
# We can use empty objects as values to force the use of
# member names instead of values ( e.g.State(1) )
class State(enum.Enum):
    Waiting = object()
    Finished = object()
    
State.Waiting, State['Finished']

(<State.Waiting: <object object at 0x7f6b0bab6040>>,
 <State.Finished: <object object at 0x7f6b0bab6050>>)

In [60]:
class AutoAlias(enum.Enum):
    def _generate_next_value_(name, start, count, last_values):
        return last_values[-1]
    
class Color(AutoAlias):
    RED = object()
    CRIMSON = enum.auto()
    CARMINE = enum.auto()
    
    BLUE = object()
    AQUAMARINE = enum.auto()
    AZURE = enum.auto()
    
Color.__members__

mappingproxy({'RED': <Color.RED: <object object at 0x7f6b0bab60c0>>,
              'CRIMSON': <Color.RED: <object object at 0x7f6b0bab60c0>>,
              'CARMINE': <Color.RED: <object object at 0x7f6b0bab60c0>>,
              'BLUE': <Color.BLUE: <object object at 0x7f6b0bab60f0>>,
              'AQUAMARINE': <Color.BLUE: <object object at 0x7f6b0bab60f0>>,
              'AZURE': <Color.BLUE: <object object at 0x7f6b0bab60f0>>})